# Undestanding Memory in LangChain

LLMs are stateless, which means they can't remember any previous information when they run. You get round this by giving them access to 'memory', or the results of previous steps. Here's some code which shows how you can implement this

https://langchain-ai.github.io/langgraph/concepts/memory/

This script sets up a simple memory-enabled chatbot using LangChain and an Ollama-hosted local language model. The chatbot can remember previous interactions in a session and use that context to provide more coherent, context-aware answers. It uses:

- LangChain's message history system to track conversation state

- A prompt template with a memory placeholder

 - An Ollama local LLM (Gemma3:1b) for generating responses

 - A session ID mechanism to maintain separate chat histories for different users or sessions

This is useful for applications like chatbots or virtual assistants that need to understand user context over multiple interactions.

---

This first cell illustrates the problem - as the LLM is stateless it doesn't remember the users name

In [2]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM

# === Step 1: Initialize the Local Language Model (LLM) ===
llm = OllamaLLM(
    model="gemma3:1b",  # Same model as the memory version
    temperature=0.6
)

# === Step 2: Define a Stateless Prompt Template ===
# This template assumes no prior history is available to the model
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}"),  # Just the current user message, no history
])

# === Step 3: Create the Stateless Chain ===
# Since we're not using memory, this is a simple pipeline
chain = prompt | llm

# === Step 4: Interact with the LLM ===
# Each call is isolated — no memory, no session state

response_1 = chain.invoke({"input": "Hey there - I'm Alistair, how are you doing today?"})
response_2 = chain.invoke({"input": "Where does the wind come from?"})
response_3 = chain.invoke({"input": "What's my name?"})  # The model won't remember "Alistair" here

# === Step 5: Print the Responses ===
print(response_1)
print(response_2)
print(response_3)


Hi Alistair! I’m doing well, thank you for asking. Just here, ready to assist. How about you? How’s your day going so far?
That’s a fantastic question! The wind comes from a few different places, but essentially it’s caused by the movement of air. Here’s a breakdown of the main sources:

* **Solar Heating:** The sun warms the Earth, and this warmth causes the air above us to expand. This expansion creates an area of lower pressure, and the warm air rises. As it rises, it cools and expands further, creating more low-pressure areas. These low-pressure areas are where the wind originates.

* **Global Wind Patterns:**  The Earth is a giant sphere, and the sun heats the equator more than the poles. This creates a zone of rising air, which then spreads out, creating large-scale wind patterns like the trade winds and jet streams.

* **Local Factors:**  Of course, wind also comes from things like:
    * **Mountains:**  Mountains can force air to rise, creating wind.
    * **Landforms:**  Flat 

In this second example, we've implemented the message history so relevant parts of the discussion are still available to the LLM

In [3]:
# Import necessary components from LangChain and Ollama
from langchain_core.chat_history import InMemoryChatMessageHistory  # In-memory storage for chat history
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  # For templating LLM prompts
from langchain_core.runnables.history import RunnableWithMessageHistory  # Chain wrapper with memory support
from langchain_ollama import OllamaLLM  # LangChain-compatible interface to a local Ollama model

# === Step 1: Initialize the Local Language Model (LLM) ===
llm = OllamaLLM(
    model="gemma3:1b",  # Using the "gemma3:1b" model via Ollama
    temperature=0.6     # Adds some creativity/randomness to the output
)

# === Step 2: Define the Prompt Template ===
# The template includes:
# - a system message giving the LLM a role
# - a placeholder for conversation history
# - the latest human input
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),                 # Static role message
    MessagesPlaceholder(variable_name="history"),               # Dynamic memory placeholder
    ("human", "{input}"),                                       # User message that gets filled in at runtime
])

# === Step 3: Create a Chain ===
# This is a pipeline where the formatted prompt feeds directly into the LLM
chain = prompt | llm

# === Step 4: Create a Store for Session Histories ===
# Each session ID will have its own in-memory message history
session_histories = {}

# === Step 5: Define a Function to Manage Session Histories ===
def get_message_history(session_id: str):
    """
    Retrieves or initializes a message history for a given session ID.
    """
    if session_id not in session_histories:
        session_histories[session_id] = InMemoryChatMessageHistory()  # New session gets a fresh history
    return session_histories[session_id]

# === Step 6: Add Memory Support to the Chain ===
# This wrapper adds automatic history tracking to the chain
with_history = RunnableWithMessageHistory(
    chain,                             # The chain to wrap
    get_message_history,              # Function to fetch session-specific memory
    input_messages_key="input",       # Key name in input dictionary for current user message
    history_messages_key="history",   # Key name in memory for storing/retrieving message history
)

# === Step 7: Simulate a Session ===
session_id = "alistair's-session"  # Simulated user/session ID

# === Step 8: Interact with the Chatbot ===
# Each interaction provides an input and session ID so memory can be used

# 1st message: Introducing the user
response_1 = with_history.invoke(
    {"input": "Hey there - I'm Alistair, how are you doing today?"},
    config={"configurable": {"session_id": session_id}}
)

# 2nd message: A follow-up question
response_2 = with_history.invoke(
    {"input": "Where does the wind come from?"},
    config={"configurable": {"session_id": session_id}}
)

# 3rd message: A memory test - LLM should remember user's name
response_3 = with_history.invoke(
    {"input": "What's my name?"},
    config={"configurable": {"session_id": session_id}}
)

# === Step 9: Display the Responses ===
print(response_1)
print(response_2)
print(response_3)

# === Optional: View Memory Contents for the Session ===
print(f"\nMemory for session '{session_id}':")
for msg in session_histories[session_id].messages:
    print(f"- {msg.type}: {msg.content}")


Hi Alistair! I’m doing well, thanks for asking. Just here, ready to assist. How about you? How’s your day going so far?
AI: That’s a great question! The wind comes from all directions, actually. It’s a result of atmospheric pressure differences. Here’s a breakdown of how it works:

*   **Warm air rises:** The sun heats the Earth, causing the warm air to rise.
*   **Descending air creates wind:** As the warm air rises, it creates an area of lower pressure. This allows cooler, denser air to rush in to fill the space.
*   **Pressure gradients:** This creates a pressure gradient – a difference in pressure – which is what drives the wind.

Essentially, the wind is a consequence of the movement of air masses, driven by temperature differences and the Earth's rotation.

Do you want to know more about how it works, or would you like me to explain it in a different way?
AI: That’s a very good question! Your name is Alistair. 😊

Memory for session 'alistair's-session':
- human: Hey there - I'm A